In [294]:
import requests
import io
import zipfile

headers = {'User-Agent':'Mozilla/5.0 \
            (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
r = requests.get\
    ('https://u626n26h74f16ig1p3pt0f2g-wpengine.netdna-ssl.com/wp-content/uploads/2016/04/Indego_Trips_2016Q1.zip', \
     headers=headers, stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))

In [295]:
import pandas as pd
df = pd.read_csv(z.open(z.namelist()[0]))

In [296]:
df['start_time'] = pd.to_datetime(df['start_time'], format='%m/%d/%Y %H:%M')
df['end_time'] = pd.to_datetime(df['end_time'], format='%m/%d/%Y %H:%M')
df.to_csv('phila-bike-share-df.csv')

In [297]:
from geopy.distance import vincenty
df = df.dropna()
for i, row in df.iterrows():
    df.loc[i, 'trip_distance'] = vincenty((row['start_lat'],row['start_lon'])
                                    ,(row['end_lat'], row['end_lon'])).miles

In [298]:
df.to_csv('phila-bike-share-distance.csv')

In [313]:
import time
from datetime import date, datetime, timedelta
from dateutil.parser import parse
datelist = generate_dates(date(2015,12,31), date(2016,4,1), timedelta(days=1))
cntr = 1 # used for a wait in WU API call
df_wu = pd.DataFrame()
for i in datelist:
    if(cntr > 10):
        time.sleep(61)
        cntr = 1
    req = \
        requests.get('http://api.wunderground.com/api/d8cbef9bf5fd26ba/history_'\
                     +str(i)+'/q/PA/Philadelphia.json')
#    print(req)
    wu_data = req.json()
    df_wu_day = pd.DataFrame.from_dict(wu_data['history']['observations'])
#     df_wu_day['date'] = pd.to_datetime(df_wu_day['date'], format='%Y-%m%-%d %H:%M:%S')
#     df_wu_day.index = [parse(df_wu_day['date']['pretty']) for n in \
#                        range(len(df_wu_day))]
    for i, rows in df_wu_day.iterrows():
        str_year = rows['date']['year']
        str_mon = rows['date']['mon']
        str_day = rows['date']['mday']
        str_hour = rows['date']['hour']
        str_min = rows['date']['min']
        obs_datetime = datetime(int(str_year), int(str_mon), int(str_day), int(str_hour), int(str_min))
        df_wu_day.loc[i, 'obs_datetime'] = obs_datetime.strftime('%Y-%m-%d %H:%M')
    df_wu = df_wu.append(df_wu_day)
    cntr += 1
df_wu['obs_datetime'] = pd.to_datetime(df_wu['obs_datetime'])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [301]:
df_wu.drop('date', axis=1, inplace=True)
df_wu.to_csv('wu-2016JantoMar-weather-nodateindex.csv')

In [72]:
def generate_dates(start, end, delta):
    datelist = []
    curr = start
    while(curr <= end):
        datelist.append(curr.strftime('%Y%m%d'))
        curr += delta
    return(datelist)

In [ ]:
import numpy as np
for i, irows in df.iterrows():
    abs_val = []
    for j, jrows in df_wu.iterrows():
        abs_val.append(abs(irows['start_time'] - pd.to_datetime(jrows['obs_datetime'])))   
    abs_dif = np.array(abs_val)
    min_index = np.argmin(abs_dif)
    df.loc[i, 'temperature'] = df_wu.iloc[min_index]['tempi']
    df.loc[i, 'condition'] = df_wu.iloc[min_index]['conds']
    df.loc[i, 'humdity'] = df_wu.iloc[min_index]['hum']
    df.loc[i, 'wind_chill'] = df_wu.iloc[min_index]['windchilli']
    df.loc[i, 'wind_speed'] = df_wu.iloc[min_index]['wspdi']

In [345]:
df_wu.iloc[0]['tempi']

'41.0'